# 운동 행동 분류 upgrade

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
%matplotlib inline
from matplotlib import rc
rc('font', family ='NanumMyeongjo')
plt.rcParams['axes.unicode_minus'] = False

train = pd.read_csv('train_features.csv')
train_label = pd.read_csv('train_labels.csv')
test = pd.read_csv('test_features.csv')
submission = pd.read_csv('sample_submission.csv')

In [3]:
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [4]:
train_label

,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)
3,3,26,Non-Exercise
4,4,26,Non-Exercise
...,...,...,...
3120,3120,26,Non-Exercise
3121,3121,26,Non-Exercise
3122,3122,15,Dynamic Stretch (at your own pace)
3123,3123,26,Non-Exercise


In [9]:
y = train_label['label']

In [14]:
X = train.drop('time',axis = 1, inplace = False).groupby('id').agg(['max','min','mean'])

In [15]:
from sklearn.model_selection import train_test_split

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [61]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [19]:
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score

In [48]:
rf = RandomForestClassifier(n_jobs = -1, random_state = 0, min_samples_leaf = 50, 
                           n_estimators = 300)

In [49]:
rf.fit(X_train, y_train)

RandomForestClassifier(min_samples_leaf=50, n_estimators=300, n_jobs=-1,
                       random_state=0)

In [50]:
rf_pred = rf.predict(X_test)
print('예측 정확도 : ', accuracy_score(y_test, rf_pred))

예측 정확도 :  0.5616


In [59]:
y_pred = rf.predict_proba(X_test)
print('log loss : ', log_loss(y_test, y_pred))

log loss :  1.6779315988933428


In [62]:
# 그래디언트부스팅 분류
gbc = GradientBoostingClassifier(random_state = 0)
gbc.fit(X_train, y_train)
y_pred = gbc.predict_proba(X_test)
print('log loss : ', log_loss(y_test, y_pred))

log loss :  2.889644006854326


GradientBoosting은 수행시간이 오래 걸리고, 이 경우 결과도 좋지 않음. 

### XGBoost

In [63]:
import xgboost as xgb

In [65]:
dtrain = xgb.DMatrix(data = X_train , label = y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)

In [78]:
xgb_params  = {
    'max_depth' : 3, 
    'eta':0.1,
    'objective':'multi:softmax',
    'num_class':61,
    'eval_metric':'mlogloss',
    'early_stoppings':100
}
num_rounds = 400

In [79]:
wlist = [(dtrain, 'train'), (dtest, 'eval')]

In [80]:
xgb_model = xgb.train(params = xgb_params,
                     dtrain = dtrain,
                     num_boost_round = num_rounds,
                     evals = wlist)

[13:05:03] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { early_stoppings } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-mlogloss:2.95622	eval-mlogloss:2.98171
[1]	train-mlogloss:2.64617	eval-mlogloss:2.73713
[2]	train-mlogloss:2.40732	eval-mlogloss:2.54863
[3]	train-mlogloss:2.21206	eval-mlogloss:2.40372
[4]	train-mlogloss:2.04704	eval-mlogloss:2.27729
[5]	train-mlogloss:1.90661	eval-mlogloss:2.17308
[6]	train-mlogloss:1.78431	eval-mlogloss:2.08873
[7]	train-mlogloss:1.67641	eval-mlogloss:2.01175
[8]	train-mlogloss:1.58236	eval-mlogloss:1.94308
[9]	train-mlogloss:1.49771	eval-mlogloss:1.88404
[10]	train-mlogloss:1.42090	eval-mlogloss:1.83218
[11]	train-mlogloss:1.35064	eval-mlogloss:1.78311
[12]	train-mlogloss:1.28672	eval-mlogloss:1.7

In [87]:
evals = [(X_test, y_test)]

In [91]:
xgb_wrapper = XGBClassifier(n_estimators = 400, learning_rate = 0.1, 
                            max_depth = 6,objective = 'multi:softprob')
                           
xgb_wrapper.fit(X_train, y_train,
                eval_metric = 'mlogloss', 
                eval_set= evals, 
                early_stopping_rounds = 100)

[0]	validation_0-mlogloss:2.98531
[1]	validation_0-mlogloss:2.72805
[2]	validation_0-mlogloss:2.53941
[3]	validation_0-mlogloss:2.38405
[4]	validation_0-mlogloss:2.25757
[5]	validation_0-mlogloss:2.15358
[6]	validation_0-mlogloss:2.06704
[7]	validation_0-mlogloss:1.99144
[8]	validation_0-mlogloss:1.92195
[9]	validation_0-mlogloss:1.86161
[10]	validation_0-mlogloss:1.80957
[11]	validation_0-mlogloss:1.76333
[12]	validation_0-mlogloss:1.71867
[13]	validation_0-mlogloss:1.67776
[14]	validation_0-mlogloss:1.64055
[15]	validation_0-mlogloss:1.60563
[16]	validation_0-mlogloss:1.57591
[17]	validation_0-mlogloss:1.54737
[18]	validation_0-mlogloss:1.52016
[19]	validation_0-mlogloss:1.49672
[20]	validation_0-mlogloss:1.47451
[21]	validation_0-mlogloss:1.45256
[22]	validation_0-mlogloss:1.43523
[23]	validation_0-mlogloss:1.41833
[24]	validation_0-mlogloss:1.40202
[25]	validation_0-mlogloss:1.38812
[26]	validation_0-mlogloss:1.37408
[27]	validation_0-mlogloss:1.36254
[28]	validation_0-mlogloss:1.3

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=400, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [122]:
from sklearn.metrics import log_loss

In [123]:
test_feature = test.drop('time',axis = 1, inplace = False).groupby('id').agg(['max','min','mean'])

In [125]:
xgb_wrapper = XGBClassifier(n_estimators = 400, learning_rate = 0.1, 
                            max_depth = 6,objective = 'multi:softprob', random_state = 0)
                           
xgb_wrapper.fit(X, y,
                eval_metric = 'mlogloss', 
                eval_set= evals, 
                early_stopping_rounds = 100)
y_pred = xgb_wrapper.predict_proba(test_feature)
submission.iloc[:,1:] = y_pred

[0]	validation_0-mlogloss:2.84060
[1]	validation_0-mlogloss:2.50411
[2]	validation_0-mlogloss:2.25167
[3]	validation_0-mlogloss:2.04681
[4]	validation_0-mlogloss:1.86809
[5]	validation_0-mlogloss:1.71801
[6]	validation_0-mlogloss:1.58655
[7]	validation_0-mlogloss:1.47160
[8]	validation_0-mlogloss:1.36873
[9]	validation_0-mlogloss:1.27684
[10]	validation_0-mlogloss:1.19176
[11]	validation_0-mlogloss:1.11716
[12]	validation_0-mlogloss:1.04906
[13]	validation_0-mlogloss:0.98478
[14]	validation_0-mlogloss:0.92580
[15]	validation_0-mlogloss:0.87323
[16]	validation_0-mlogloss:0.82283
[17]	validation_0-mlogloss:0.77608
[18]	validation_0-mlogloss:0.73274
[19]	validation_0-mlogloss:0.69288
[20]	validation_0-mlogloss:0.65498
[21]	validation_0-mlogloss:0.62087
[22]	validation_0-mlogloss:0.58857
[23]	validation_0-mlogloss:0.55738
[24]	validation_0-mlogloss:0.52777
[25]	validation_0-mlogloss:0.50193
[26]	validation_0-mlogloss:0.47757
[27]	validation_0-mlogloss:0.45386
[28]	validation_0-mlogloss:0.4

In [126]:
submission.to_csv('submission파일/xgb_1.csv', index =False)

제출 결과값 : 1.13185

### LightGBM

In [111]:
X.columns = ["_".join(pair) for pair in X.columns]

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [113]:
X_train

,acc_x_max,acc_x_min,acc_x_mean,acc_y_max,acc_y_min,acc_y_mean,acc_z_max,acc_z_min,acc_z_mean,gy_x_max,gy_x_min,gy_x_mean,gy_y_max,gy_y_min,gy_y_mean,gy_z_max,gy_z_min,gy_z_mean
id,,,,,,,,,,,,,,,,,,
637,0.630341,-1.553586,-0.914852,0.481758,-1.345256,-0.219362,0.543381,-0.621472,0.079205,155.432244,-116.416430,19.392487,122.501065,-63.337940,10.558328,155.769144,-83.379569,5.476122
2495,1.229183,-1.267183,-0.062973,0.705641,-1.297130,-0.482426,1.023078,-0.553890,0.187866,134.989111,-139.410780,-0.297361,249.196681,-215.248453,-6.905798,228.195810,-193.223855,8.437690
2269,-0.359889,-1.330484,-0.955888,0.167601,-0.386703,-0.144045,-0.041769,-0.800262,-0.434122,62.479261,-63.112405,-1.368512,99.885034,-71.549489,4.119046,45.820292,-43.546738,-0.820869
2758,1.696315,-4.202443,-0.700217,1.879693,-1.602517,-0.222679,1.593441,-1.420791,0.184293,259.536133,-285.646546,-30.890240,378.049913,-165.060297,11.284496,346.540460,-214.912692,8.336780
869,-0.199650,-1.248644,-0.862834,0.380537,-0.732489,0.044240,0.277591,-1.055990,-0.152600,148.920413,-68.921948,-10.097804,137.870339,-96.720419,-0.647666,82.062896,-126.252451,2.652288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3092,1.581188,-4.654716,-0.855352,2.960999,-2.464892,0.754499,1.780154,-1.902666,0.004954,366.938885,-429.495378,23.551214,409.474732,-199.387241,26.874102,419.450969,-372.486775,0.677226
1095,1.236061,0.760399,0.957932,0.576283,0.029909,0.277585,0.328786,-0.057424,0.159487,23.164800,-86.858247,-4.640970,23.825610,-35.504522,-2.607308,34.769856,-46.655736,-0.459161
1130,0.856484,-1.382076,-0.314649,0.160486,-2.299112,-0.572248,0.929474,-0.503846,0.046407,284.929849,-358.445381,-8.805036,135.356984,-128.128116,-6.465130,190.217861,-186.185652,7.925878


In [118]:
lgbm_wrapper = LGBMClassifier(n_estimators = 400, learning_rate = 0.05, random_state = 0)

In [115]:
evals = [(X_test, y_test)]

In [119]:
lgbm_wrapper.fit(X_train,y_train,eval_set = evals, 
                early_stopping_rounds = 100)

[1]	valid_0's multi_logloss: 2.14459
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 2.01635
[3]	valid_0's multi_logloss: 1.92241
[4]	valid_0's multi_logloss: 1.85476
[5]	valid_0's multi_logloss: 1.79347
[6]	valid_0's multi_logloss: 1.74316
[7]	valid_0's multi_logloss: 1.69772
[8]	valid_0's multi_logloss: 1.6598
[9]	valid_0's multi_logloss: 1.62509
[10]	valid_0's multi_logloss: 1.59701
[11]	valid_0's multi_logloss: 1.56994
[12]	valid_0's multi_logloss: 1.5432
[13]	valid_0's multi_logloss: 1.52099
[14]	valid_0's multi_logloss: 1.50081
[15]	valid_0's multi_logloss: 1.48382
[16]	valid_0's multi_logloss: 1.46762
[17]	valid_0's multi_logloss: 1.45267
[18]	valid_0's multi_logloss: 1.43712
[19]	valid_0's multi_logloss: 1.42554
[20]	valid_0's multi_logloss: 1.413
[21]	valid_0's multi_logloss: 1.40192
[22]	valid_0's multi_logloss: 1.39003
[23]	valid_0's multi_logloss: 1.38049
[24]	valid_0's multi_logloss: 1.37343
[25]	valid_0's multi_logloss: 1.36833
[

LGBMClassifier(learning_rate=0.05, n_estimators=400, random_state=0)

In [127]:
lgbm_wrapper.fit(X,y,eval_set = evals, 
                early_stopping_rounds = 100)

[1]	valid_0's multi_logloss: 1.80527
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 1.56318
[3]	valid_0's multi_logloss: 1.38595
[4]	valid_0's multi_logloss: 1.2482
[5]	valid_0's multi_logloss: 1.12826
[6]	valid_0's multi_logloss: 1.02301
[7]	valid_0's multi_logloss: 0.931957
[8]	valid_0's multi_logloss: 0.854426
[9]	valid_0's multi_logloss: 0.786562
[10]	valid_0's multi_logloss: 0.72418
[11]	valid_0's multi_logloss: 0.670512
[12]	valid_0's multi_logloss: 0.621301
[13]	valid_0's multi_logloss: 0.576323
[14]	valid_0's multi_logloss: 0.535805
[15]	valid_0's multi_logloss: 0.497814
[16]	valid_0's multi_logloss: 0.463512
[17]	valid_0's multi_logloss: 0.431284
[18]	valid_0's multi_logloss: 0.402633
[19]	valid_0's multi_logloss: 0.376039
[20]	valid_0's multi_logloss: 0.351028
[21]	valid_0's multi_logloss: 0.32806
[22]	valid_0's multi_logloss: 0.306902
[23]	valid_0's multi_logloss: 0.287342
[24]	valid_0's multi_logloss: 0.268733
[25]	valid_0's multi

LGBMClassifier(learning_rate=0.05, n_estimators=400, random_state=0)

In [128]:
y_pred = lgbm_wrapper.predict_proba(test_feature)
submission.iloc[:,1:] = y_pred
submission.to_csv('submission파일/lgbm_1.csv', index =False)

제출 결과값 : 1.8358279907